In [65]:
import os, sys, email,re
import numpy as np 
import pandas as pd
# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set_style('whitegrid')
import wordcloud

# Network analysis
import networkx as nx

# NLP
from nltk.tokenize.regexp import RegexpTokenizer

from subprocess import check_output

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
#from sklearn.lda import LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import LatentDirichletAllocation

import gensim
from gensim import corpora
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.stem.porter import PorterStemmer

In [66]:
emails_df = pd.read_csv(r'**\emails.csv')
print(emails_df.shape)
emails_df.head()

(517401, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [67]:
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [68]:
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)
# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages

emails_df.head()

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",(phillip.allen@enron.com),(tim.belden@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",(phillip.allen@enron.com),(leah.arsdall@enron.com),Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",(phillip.allen@enron.com),(randall.gay@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p


In [69]:
emails_df = emails_df.set_index('Message-ID')\
    .drop(['file', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding'], axis=1)
# Parse datetime
emails_df['Date'] = pd.to_datetime(emails_df['Date'], infer_datetime_format=True)
emails_df.dtypes

Date          object
From          object
To            object
Subject       object
X-From        object
X-To          object
X-cc          object
X-bcc         object
X-Folder      object
X-Origin      object
X-FileName    object
content       object
user          object
dtype: object

In [70]:
import glob
list_found = {}
cats = []
for path in glob.glob("enron_with_categories/*/*.txt"):
    #print(path)
    splitPath = path.split("\\")
    batch, filename = splitPath[1:]
    contents = open(path, "r").read()

    try:
        email_parsed = email.message_from_string(contents)
        list_found[email_parsed['Message-ID']] = [x.split(',') 
                                                  for x in 
                                                  open(path.replace(".txt", ".cats")).read().split()]
    except Exception as e:
        print("error: {}".format(e))

In [71]:
for x in range(12):
    x += 1
    emails_df['Cat_{}_level_1'.format(x)] = None
    emails_df['Cat_{}_level_2'.format(x)] = None
    emails_df['Cat_{}_weight'.format(x)] = None

In [72]:

emails_df.columns

Index(['Date', 'From', 'To', 'Subject', 'X-From', 'X-To', 'X-cc', 'X-bcc',
       'X-Folder', 'X-Origin', 'X-FileName', 'content', 'user',
       'Cat_1_level_1', 'Cat_1_level_2', 'Cat_1_weight', 'Cat_2_level_1',
       'Cat_2_level_2', 'Cat_2_weight', 'Cat_3_level_1', 'Cat_3_level_2',
       'Cat_3_weight', 'Cat_4_level_1', 'Cat_4_level_2', 'Cat_4_weight',
       'Cat_5_level_1', 'Cat_5_level_2', 'Cat_5_weight', 'Cat_6_level_1',
       'Cat_6_level_2', 'Cat_6_weight', 'Cat_7_level_1', 'Cat_7_level_2',
       'Cat_7_weight', 'Cat_8_level_1', 'Cat_8_level_2', 'Cat_8_weight',
       'Cat_9_level_1', 'Cat_9_level_2', 'Cat_9_weight', 'Cat_10_level_1',
       'Cat_10_level_2', 'Cat_10_weight', 'Cat_11_level_1', 'Cat_11_level_2',
       'Cat_11_weight', 'Cat_12_level_1', 'Cat_12_level_2', 'Cat_12_weight'],
      dtype='object')

In [73]:
emails_df['labeled'] = False       
for item, val in list_found.items():
    emails_df.loc[item, 'labeled'] = True
    i = 0
    for lev1, lev2, weight in val:
        i += 1
        emails_df.loc[item, 'Cat_{}_level_1'.format(i)] = lev1
        emails_df.loc[item, 'Cat_{}_level_2'.format(i)] = lev2
        emails_df.loc[item, 'Cat_{}_weight'.format(i)] = weight

In [74]:
emails_df.columns

Index(['Date', 'From', 'To', 'Subject', 'X-From', 'X-To', 'X-cc', 'X-bcc',
       'X-Folder', 'X-Origin', 'X-FileName', 'content', 'user',
       'Cat_1_level_1', 'Cat_1_level_2', 'Cat_1_weight', 'Cat_2_level_1',
       'Cat_2_level_2', 'Cat_2_weight', 'Cat_3_level_1', 'Cat_3_level_2',
       'Cat_3_weight', 'Cat_4_level_1', 'Cat_4_level_2', 'Cat_4_weight',
       'Cat_5_level_1', 'Cat_5_level_2', 'Cat_5_weight', 'Cat_6_level_1',
       'Cat_6_level_2', 'Cat_6_weight', 'Cat_7_level_1', 'Cat_7_level_2',
       'Cat_7_weight', 'Cat_8_level_1', 'Cat_8_level_2', 'Cat_8_weight',
       'Cat_9_level_1', 'Cat_9_level_2', 'Cat_9_weight', 'Cat_10_level_1',
       'Cat_10_level_2', 'Cat_10_weight', 'Cat_11_level_1', 'Cat_11_level_2',
       'Cat_11_weight', 'Cat_12_level_1', 'Cat_12_level_2', 'Cat_12_weight',
       'labeled'],
      dtype='object')

In [75]:
emails_df[emails_df['labeled'] == True]

,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,...,Cat_10_level_1,Cat_10_level_2,Cat_10_weight,Cat_11_level_1,Cat_11_level_2,Cat_11_weight,Cat_12_level_1,Cat_12_level_2,Cat_12_weight,labeled
Message-ID,,,,,,,,,,,,,,,,,,,,,
<9831685.1075855725804.JavaMail.evans@thyme>,2001-03-15 06:45:00-08:00,(phillip.allen@enron.com),(todd.burke@enron.com),Re: Confidential Employee Information/Lenhart,Phillip K Allen,Todd Burke,,,\Phillip_Allen_June2001\Notes Folders\'sent mail,Allen-P,...,None,None,None,None,None,None,None,None,None,True
<21041312.1075855725847.JavaMail.evans@thyme>,2001-03-15 06:11:00-08:00,(phillip.allen@enron.com),(kim.bolton@enron.com),RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Phillip K Allen,Kim Bolton,,,\Phillip_Allen_June2001\Notes Folders\'sent mail,Allen-P,...,None,None,None,None,None,None,None,None,None,True
<5907100.1075858639941.JavaMail.evans@thyme>,2001-06-20 10:04:51-07:00,(k..allen@enron.com),"(matthew.lenhart@enron.com, jay.reitmeyer@enro...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",,,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",Allen-P,...,None,None,None,None,None,None,None,None,None,True
<26625142.1075858639964.JavaMail.evans@thyme>,2001-06-20 10:09:00-07:00,(k..allen@enron.com),"(matthew.lenhart@enron.com, jay.reitmeyer@enro...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",,,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",Allen-P,...,None,None,None,None,None,None,None,None,None,True
<19730598.1075858642129.JavaMail.evans@thyme>,2001-08-09 05:30:58-07:00,(k..allen@enron.com),"(matt.smith@enron.com, m..tholt@enron.com)",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Smith, Matt </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",,,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",Allen-P,...,None,None,None,None,None,None,None,None,None,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
<18361957.1075861368310.JavaMail.evans@thyme>,2001-11-05 05:42:45-08:00,(louise.kitchen@enron.com),"(john.lavorato@enron.com, w..delainey@enron.co...",Confidential Due Diligence Info - GS,"Kitchen, Louise </O=ENRON/OU=NA/CN=RECIPIENTS/...","Whalley, Greg </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",,,"\GWHALLE (Non-Privileged)\Whalley, Greg\Inbox",Whalley-G,...,None,None,None,None,None,None,None,None,None,True
<29049822.1075842045435.JavaMail.evans@thyme>,2001-06-13 07:00:00-07:00,(steven.kean@enron.com),"(jeff.skilling@enron.com, james.derrick@enron....",California's dim bulbs,Steven J Kean,"Kenneth Lay, Jeff Skilling, Andrew S Fastow, G...",,,\Lawrence_Whalley_Nov2001\Notes Folders\All do...,WHALLEY-G,...,None,None,None,None,None,None,None,None,None,True
<205897.1075861997314.JavaMail.evans@thyme>,2001-11-13 14:20:01-08:00,(jeanie.slone@enron.com),(mark.whitt@enron.com),RE: confidential ee info,"Slone, Jeanie </O=ENRON/OU=NA/CN=RECIPIENTS/CN...","Whitt, Mark </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",,,"\MWHITT (Non-Privileged)\Whitt, Mark\Inbox",Whitt-M,...,None,None,None,None,None,None,None,None,None,True


In [12]:

len(emails_df.columns)

50

In [80]:
email_df_labelled = emails_df[emails_df['labeled'] == True]

In [13]:
emails_df.reset_index(level=0, inplace=True)

In [14]:
emails_df['content']

0                                 Here is our forecast\n\n 
1         Traveling to have a business meeting takes the...
2                            test successful.  way to go!!!
3         Randy,\n\n Can you send me a schedule of the s...
4                       Let's shoot for Tuesday at 11:45.  
                                ...                        
517396    This is a trade with OIL-SPEC-HEDGE-NG (John L...
517397    Some of my position is with the Alberta Term b...
517398    2\n\n -----Original Message-----\nFrom: \tDouc...
517399    Analyst\t\t\t\t\tRank\n\nStephane Brodeur\t\t\...
517400    i think the YMCA has a class that is for peopl...
Name: content, Length: 517401, dtype: object

In [15]:
import re

def remove_reg_expressions(row):
    tokens = []
    try:
        for token in row:
            token = token.lower()
            token = re.sub(r'[\W\d]', " ", token)
            tokens.append(token)
    except:
        token = ""
        tokens.append(token)
    return tokens
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')    

def stop_word_removal(row):
    token = [token for token in row if token not in stopwords]
    token = filter(None, token)
    return token
def tokenize(row):
    if row is None or row is '':
        tokens = ""
    else:
        tokens = row.split(" ")
    return tokens

def returnCleanText(row):
    #return "".join(remove_reg_expressions(tokenize(row)))
    return " ".join(remove_reg_expressions(tokenize(row)))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ag23489\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
emails_df['labeled'].value_counts()

False    515699
True       1702
Name: labeled, dtype: int64

In [17]:
email_df_labelled = emails_df[emails_df['labeled']]

In [18]:
email_df_labelled.reset_index(inplace=True,drop = True)

In [19]:
email_df_labelled['labels'] = np.nan

C:\Users\ag23489\.conda\envs\mypy36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
"""
labels
"""
labelNames = pd.read_excel(r'C:\Users\ag23489\Documents\enronEmailLabels.xlsx')

In [21]:
labelNames['Category']
categories = []
namesofcategoris = []
for i in labelNames['Category']:
    categories.append(format(i,".2f"))
for i in labelNames['Name']:
    namesofcategoris.append(i)
namesofcategoris

['Company Business, Strategy',
 'Purely Personal',
 'Personal but in professional context',
 'Logistic Arrangements',
 'Employment arrangements',
 'Document editing/checking ',
 'Empty message Attachment',
 'Empty message',
 'Includes new text in addition to forwarded material',
 'Forwarded email(s) including replies',
 'Business letter/Document',
 'News article(s)',
 'Government / academic report(s)',
 'Government action(s)',
 'Press release(s)',
 'Legal documents',
 'Pointers to url(s)',
 'Newsletters',
 'JokesBusiness',
 'JokesPersonal',
 'Attachment(s)',
 'regulations and regulators',
 'internal projects',
 'company image',
 'company image',
 'political influence',
 'california energy crisis',
 'internal company policy',
 'internal company operations',
 'alliances',
 'legal advice',
 'talking points',
 'meeting minutes',
 'trip reports',
 'jubilation',
 'hope',
 'humor',
 'camaraderie',
 'admiration',
 'gratitude',
 'friendship',
 'sympathy',
 'sarcasm',
 'confidentiality',
 'anxie

In [22]:
labelNamesDict = dict(zip(categories,namesofcategoris))

In [23]:
labelNamesDict

{'1.10': 'Company Business, Strategy',
 '1.20': 'Purely Personal',
 '1.30': 'Personal but in professional context',
 '1.40': 'Logistic Arrangements',
 '1.50': 'Employment arrangements',
 '1.60': 'Document editing/checking ',
 '1.70': 'Empty message Attachment',
 '1.80': 'Empty message',
 '2.10': 'Newsletters',
 '2.20': 'Forwarded email(s) including replies',
 '2.30': 'Business letter/Document',
 '2.40': 'News article(s)',
 '2.50': 'Government / academic report(s)',
 '2.60': 'Government action(s)',
 '2.70': 'Press release(s)',
 '2.80': 'Legal documents',
 '2.90': 'Pointers to url(s)',
 '2.11': 'JokesBusiness',
 '2.12': 'JokesPersonal',
 '2.13': 'Attachment(s)',
 '3.10': 'legal advice',
 '3.20': 'internal projects',
 '3.30': 'company image',
 '3.40': 'company image',
 '3.50': 'political influence',
 '3.60': 'california energy crisis',
 '3.70': 'internal company policy',
 '3.80': 'internal company operations',
 '3.90': 'alliances',
 '3.11': 'talking points',
 '3.12': 'meeting minutes',
 '

In [24]:
email_df_labelled[email_df_labelled['Cat_8_level_1'].notnull()]['Cat_3_level_1']

65      2
151     2
284     2
391     1
458     2
546     3
937     2
970     2
1232    2
1363    2
1501    2
1541    2
1554    3
1611    2
1639    2
Name: Cat_3_level_1, dtype: object

In [25]:
labels = []
from numpy import nan
for index,row in email_df_labelled.iterrows():
    print(row[14:-2])
    labelsList = row[14:-2]
    labelsList = [x for x in labelsList if str(x) != 'None']
    print(labelsList)
    #print(len(labelsList))
    labeltoappend = []
    for i in range(2,len(labelsList),3):
        jointString = labelsList[i-2]+'.'+labelsList[i-1]
        
        labeltoappend.append(format(float(jointString),".2f"))
    print(labeltoappend)
    labels.append(labeltoappend)
    

Cat_1_level_1        1
Cat_1_level_2        1
Cat_1_weight         2
Cat_2_level_1        3
Cat_2_level_2        8
Cat_2_weight         2
Cat_3_level_1     None
Cat_3_level_2     None
Cat_3_weight      None
Cat_4_level_1     None
Cat_4_level_2     None
Cat_4_weight      None
Cat_5_level_1     None
Cat_5_level_2     None
Cat_5_weight      None
Cat_6_level_1     None
Cat_6_level_2     None
Cat_6_weight      None
Cat_7_level_1     None
Cat_7_level_2     None
Cat_7_weight      None
Cat_8_level_1     None
Cat_8_level_2     None
Cat_8_weight      None
Cat_9_level_1     None
Cat_9_level_2     None
Cat_9_weight      None
Cat_10_level_1    None
Cat_10_level_2    None
Cat_10_weight     None
Cat_11_level_1    None
Cat_11_level_2    None
Cat_11_weight     None
Cat_12_level_1    None
Cat_12_level_2    None
Cat_12_weight     None
Name: 0, dtype: object
['1', '1', '2', '3', '8', '2']
['1.10', '3.80']
Cat_1_level_1        1
Cat_1_level_2        3
Cat_1_weight         2
Cat_2_level_1        4
Cat_2_lev

Name: 129, dtype: object
['1', '4', '2', '3', '8', '1']
['1.40', '3.80']
Cat_1_level_1        1
Cat_1_level_2        1
Cat_1_weight         2
Cat_2_level_1        2
Cat_2_level_2        4
Cat_2_weight         2
Cat_3_level_1        3
Cat_3_level_2        3
Cat_3_weight         1
Cat_4_level_1     None
Cat_4_level_2     None
Cat_4_weight      None
Cat_5_level_1     None
Cat_5_level_2     None
Cat_5_weight      None
Cat_6_level_1     None
Cat_6_level_2     None
Cat_6_weight      None
Cat_7_level_1     None
Cat_7_level_2     None
Cat_7_weight      None
Cat_8_level_1     None
Cat_8_level_2     None
Cat_8_weight      None
Cat_9_level_1     None
Cat_9_level_2     None
Cat_9_weight      None
Cat_10_level_1    None
Cat_10_level_2    None
Cat_10_weight     None
Cat_11_level_1    None
Cat_11_level_2    None
Cat_11_weight     None
Cat_12_level_1    None
Cat_12_level_2    None
Cat_12_weight     None
Name: 130, dtype: object
['1', '1', '2', '2', '4', '2', '3', '3', '1']
['1.10', '2.40', '3.30']
Cat

Name: 272, dtype: object
['1', '1', '2', '2', '2', '2', '3', '2', '2']
['1.10', '2.20', '3.20']
Cat_1_level_1        1
Cat_1_level_2        3
Cat_1_weight         2
Cat_2_level_1        3
Cat_2_level_2        5
Cat_2_weight         2
Cat_3_level_1        4
Cat_3_level_2       19
Cat_3_weight         1
Cat_4_level_1     None
Cat_4_level_2     None
Cat_4_weight      None
Cat_5_level_1     None
Cat_5_level_2     None
Cat_5_weight      None
Cat_6_level_1     None
Cat_6_level_2     None
Cat_6_weight      None
Cat_7_level_1     None
Cat_7_level_2     None
Cat_7_weight      None
Cat_8_level_1     None
Cat_8_level_2     None
Cat_8_weight      None
Cat_9_level_1     None
Cat_9_level_2     None
Cat_9_weight      None
Cat_10_level_1    None
Cat_10_level_2    None
Cat_10_weight     None
Cat_11_level_1    None
Cat_11_level_2    None
Cat_11_weight     None
Cat_12_level_1    None
Cat_12_level_2    None
Cat_12_weight     None
Name: 273, dtype: object
['1', '3', '2', '3', '5', '2', '4', '19', '1']
['1.

['1', '4', '2', '1', '6', '1']
['1.40', '1.60']
Cat_1_level_1        1
Cat_1_level_2        3
Cat_1_weight         1
Cat_2_level_1        1
Cat_2_level_2        4
Cat_2_weight         2
Cat_3_level_1     None
Cat_3_level_2     None
Cat_3_weight      None
Cat_4_level_1     None
Cat_4_level_2     None
Cat_4_weight      None
Cat_5_level_1     None
Cat_5_level_2     None
Cat_5_weight      None
Cat_6_level_1     None
Cat_6_level_2     None
Cat_6_weight      None
Cat_7_level_1     None
Cat_7_level_2     None
Cat_7_weight      None
Cat_8_level_1     None
Cat_8_level_2     None
Cat_8_weight      None
Cat_9_level_1     None
Cat_9_level_2     None
Cat_9_weight      None
Cat_10_level_1    None
Cat_10_level_2    None
Cat_10_weight     None
Cat_11_level_1    None
Cat_11_level_2    None
Cat_11_weight     None
Cat_12_level_1    None
Cat_12_level_2    None
Cat_12_weight     None
Name: 397, dtype: object
['1', '3', '1', '1', '4', '2']
['1.30', '1.40']
Cat_1_level_1        1
Cat_1_level_2        4
Cat_1

['1.10', '2.20', '2.60', '3.50', '3.80']
Cat_1_level_1        1
Cat_1_level_2        1
Cat_1_weight         2
Cat_2_level_1        2
Cat_2_level_2        1
Cat_2_weight         2
Cat_3_level_1        2
Cat_3_level_2        4
Cat_3_weight         2
Cat_4_level_1        3
Cat_4_level_2        1
Cat_4_weight         2
Cat_5_level_1        3
Cat_5_level_2        6
Cat_5_weight         2
Cat_6_level_1     None
Cat_6_level_2     None
Cat_6_weight      None
Cat_7_level_1     None
Cat_7_level_2     None
Cat_7_weight      None
Cat_8_level_1     None
Cat_8_level_2     None
Cat_8_weight      None
Cat_9_level_1     None
Cat_9_level_2     None
Cat_9_weight      None
Cat_10_level_1    None
Cat_10_level_2    None
Cat_10_weight     None
Cat_11_level_1    None
Cat_11_level_2    None
Cat_11_weight     None
Cat_12_level_1    None
Cat_12_level_2    None
Cat_12_weight     None
Name: 566, dtype: object
['1', '1', '2', '2', '1', '2', '2', '4', '2', '3', '1', '2', '3', '6', '2']
['1.10', '2.10', '2.40', '3.10

...

Name: 1007, dtype: object
['1', '1', '2', '2', '2', '2', '3', '10', '2']
['1.10', '2.20', '3.10']
Cat_1_level_1        1
Cat_1_level_2        1
Cat_1_weight         2
Cat_2_level_1        2
Cat_2_level_2        2
Cat_2_weight         2
Cat_3_level_1        3
Cat_3_level_2        1
Cat_3_weight         2
Cat_4_level_1        3
Cat_4_level_2        7
Cat_4_weight         2
Cat_5_level_1        4
Cat_5_level_2        4
Cat_5_weight         2
Cat_6_level_1     None
Cat_6_level_2     None
Cat_6_weight      None
Cat_7_level_1     None
Cat_7_level_2     None
Cat_7_weight      None
Cat_8_level_1     None
Cat_8_level_2     None
Cat_8_weight      None
Cat_9_level_1     None
Cat_9_level_2     None
Cat_9_weight      None
Cat_10_level_1    None
Cat_10_level_2    None
Cat_10_weight     None
Cat_11_level_1    None
Cat_11_level_2    None
Cat_11_weight     None
Cat_12_level_1    None
Cat_12_level_2    None
Cat_12_weight     None
Name: 1008, dtype: object
['1', '1', '2', '2', '2', '2', '3', '1', '2', '3

Name: 1155, dtype: object
['1', '4', '1']
['1.40']
Cat_1_level_1        1
Cat_1_level_2        6
Cat_1_weight         2
Cat_2_level_1        2
Cat_2_level_2        1
Cat_2_weight         2
Cat_3_level_1        2
Cat_3_level_2        2
Cat_3_weight         2
Cat_4_level_1     None
Cat_4_level_2     None
Cat_4_weight      None
Cat_5_level_1     None
Cat_5_level_2     None
Cat_5_weight      None
Cat_6_level_1     None
Cat_6_level_2     None
Cat_6_weight      None
Cat_7_level_1     None
Cat_7_level_2     None
Cat_7_weight      None
Cat_8_level_1     None
Cat_8_level_2     None
Cat_8_weight      None
Cat_9_level_1     None
Cat_9_level_2     None
Cat_9_weight      None
Cat_10_level_1    None
Cat_10_level_2    None
Cat_10_weight     None
Cat_11_level_1    None
Cat_11_level_2    None
Cat_11_weight     None
Cat_12_level_1    None
Cat_12_level_2    None
Cat_12_weight     None
Name: 1156, dtype: object
['1', '6', '2', '2', '1', '2', '2', '2', '2']
['1.60', '2.10', '2.20']
Cat_1_level_1        1
C

['1', '4', '2', '2', '2', '2']
['1.40', '2.20']
Cat_1_level_1        1
Cat_1_level_2        1
Cat_1_weight         2
Cat_2_level_1        3
Cat_2_level_2        5
Cat_2_weight         2
Cat_3_level_1     None
Cat_3_level_2     None
Cat_3_weight      None
Cat_4_level_1     None
Cat_4_level_2     None
Cat_4_weight      None
Cat_5_level_1     None
Cat_5_level_2     None
Cat_5_weight      None
Cat_6_level_1     None
Cat_6_level_2     None
Cat_6_weight      None
Cat_7_level_1     None
Cat_7_level_2     None
Cat_7_weight      None
Cat_8_level_1     None
Cat_8_level_2     None
Cat_8_weight      None
Cat_9_level_1     None
Cat_9_level_2     None
Cat_9_weight      None
Cat_10_level_1    None
Cat_10_level_2    None
Cat_10_weight     None
Cat_11_level_1    None
Cat_11_level_2    None
Cat_11_weight     None
Cat_12_level_1    None
Cat_12_level_2    None
Cat_12_weight     None
Name: 1322, dtype: object
['1', '1', '2', '3', '5', '2']
['1.10', '3.50']
Cat_1_level_1        1
Cat_1_level_2        1
Cat_

['1', '1', '2', '2', '1', '2', '2', '2', '2', '2', '8', '1', '2', '13', '2', '3', '10', '2', '4', '10', '2']
['1.10', '2.10', '2.20', '2.80', '2.13', '3.10', '4.10']
Cat_1_level_1        1
Cat_1_level_2        1
Cat_1_weight         2
Cat_2_level_1        3
Cat_2_level_2        6
Cat_2_weight         2
Cat_3_level_1        3
Cat_3_level_2       12
Cat_3_weight         2
Cat_4_level_1     None
Cat_4_level_2     None
Cat_4_weight      None
Cat_5_level_1     None
Cat_5_level_2     None
Cat_5_weight      None
Cat_6_level_1     None
Cat_6_level_2     None
Cat_6_weight      None
Cat_7_level_1     None
Cat_7_level_2     None
Cat_7_weight      None
Cat_8_level_1     None
Cat_8_level_2     None
Cat_8_weight      None
Cat_9_level_1     None
Cat_9_level_2     None
Cat_9_weight      None
Cat_10_level_1    None
Cat_10_level_2    None
Cat_10_weight     None
Cat_11_level_1    None
Cat_11_level_2    None
Cat_11_weight     None
Cat_12_level_1    None
Cat_12_level_2    None
Cat_12_weight     None
Name: 

['1', '1', '2', '2', '1', '1', '2', '2', '1', '3', '7', '2', '4', '10', '2']
['1.10', '2.10', '2.20', '3.70', '4.10']
Cat_1_level_1        1
Cat_1_level_2        1
Cat_1_weight         2
Cat_2_level_1        2
Cat_2_level_2        2
Cat_2_weight         1
Cat_3_level_1        3
Cat_3_level_2        6
Cat_3_weight         2
Cat_4_level_1        3
Cat_4_level_2       10
Cat_4_weight         2
Cat_5_level_1     None
Cat_5_level_2     None
Cat_5_weight      None
Cat_6_level_1     None
Cat_6_level_2     None
Cat_6_weight      None
Cat_7_level_1     None
Cat_7_level_2     None
Cat_7_weight      None
Cat_8_level_1     None
Cat_8_level_2     None
Cat_8_weight      None
Cat_9_level_1     None
Cat_9_level_2     None
Cat_9_weight      None
Cat_10_level_1    None
Cat_10_level_2    None
Cat_10_weight     None
Cat_11_level_1    None
Cat_11_level_2    None
Cat_11_weight     None
Cat_12_level_1    None
Cat_12_level_2    None
Cat_12_weight     None
Name: 1672, dtype: object
['1', '1', '2', '2', '2', '1

In [27]:
#multiCoarseCount
multiCoarse = 0
maxCoarse1 = 0
maxCoarse = 0
for i in labels:
    if sum('1.' in j for j in i)>1:
        
        maxCoarse = sum('1.' in j for j in i)
        if maxCoarse>maxCoarse1:
            maxCoarse1 = maxCoarse
        multiCoarse += 1
        
        

In [28]:
multiCoarse
maxCoarse1

3

In [29]:
email_df_labelled.columns

Index(['Message-ID', 'Date', 'From', 'To', 'Subject', 'X-From', 'X-To', 'X-cc',
       'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'content', 'user',
       'Cat_1_level_1', 'Cat_1_level_2', 'Cat_1_weight', 'Cat_2_level_1',
       'Cat_2_level_2', 'Cat_2_weight', 'Cat_3_level_1', 'Cat_3_level_2',
       'Cat_3_weight', 'Cat_4_level_1', 'Cat_4_level_2', 'Cat_4_weight',
       'Cat_5_level_1', 'Cat_5_level_2', 'Cat_5_weight', 'Cat_6_level_1',
       'Cat_6_level_2', 'Cat_6_weight', 'Cat_7_level_1', 'Cat_7_level_2',
       'Cat_7_weight', 'Cat_8_level_1', 'Cat_8_level_2', 'Cat_8_weight',
       'Cat_9_level_1', 'Cat_9_level_2', 'Cat_9_weight', 'Cat_10_level_1',
       'Cat_10_level_2', 'Cat_10_weight', 'Cat_11_level_1', 'Cat_11_level_2',
       'Cat_11_weight', 'Cat_12_level_1', 'Cat_12_level_2', 'Cat_12_weight',
       'labeled', 'labels'],
      dtype='object')

In [30]:
for i in labelNames['Name']:
    email_df_labelled[i] = 0

C:\Users\ag23489\.conda\envs\mypy36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
email_df_labelled.columns   

Index(['Message-ID', 'Date', 'From', 'To', 'Subject', 'X-From', 'X-To', 'X-cc',
       'X-bcc', 'X-Folder',
       ...
       'confidentiality', 'anxiety', 'concern', 'competitiveness', 'gloating',
       'pride', 'agitation', 'despair', 'shame', 'dislike'],
      dtype='object', length=104)

In [32]:
labelsWithNames = []
for i in labels:
    labelsWithNames.append(list(map(labelNamesDict.get, i)))

In [33]:
labels
for i in labels:
    onePoint = False
    threePoint = False
    for j in i:
        if '1.' in j:
            onePoint = True
    for j in i:
        if '3.' in j:
            threePoint = True
    if onePoint and not threePoint:
        print(i)

['1.30', '4.10', '4.11']
['1.40', '2.20']
['1.40', '2.20']
['1.40', '2.20']
['1.40', '2.10', '2.20']
['1.50', '2.10', '2.20', '4.60', '4.10']
['1.20', '4.60']
['1.30']
['1.20']
['1.30', '2.10', '2.11', '4.30', '4.90']
['1.40']
['1.40']
['1.40', '2.90']
['1.50', '2.13', '4.10']
['1.10', '1.40', '2.20', '4.10']
['1.70', '2.13']
['1.70', '2.10', '2.13']
['1.40', '2.10', '2.20']
['1.60', '2.10', '2.20']
['1.50', '1.60', '2.20']
['1.50', '2.20']
['1.50', '2.10', '2.20']
['1.40']
['1.50', '4.10']
['1.10', '2.13', '4.10', '4.12']
['1.30', '2.10', '2.20', '2.13', '4.11']
['1.10', '4.10']
['1.50', '2.10', '2.20']
['1.30', '4.30']
['1.30', '2.20', '4.16']
['1.40']
['1.60', '2.13']
['1.40', '2.10', '2.20', '2.40']
['1.40', '2.10', '2.20']
['1.30', '4.11']
['1.40', '2.20']
['1.40', '2.20']
['1.30', '1.40', '2.10', '2.20', '4.30', '4.90']
['1.60', '2.20']
['1.10', '2.10']
['1.40']
['1.40', '4.90']
['1.60', '2.10', '2.20', '2.13']
['1.10', '2.20']
['1.70', '2.13']
['1.10', '1.40', '2.13']
['1.40', '

In [34]:
for index,row in email_df_labelled.iterrows():
    for i in labelsWithNames[index]:
        print(index,i)
        email_df_labelled.loc[index,i] = 1

0 Company Business, Strategy
0 internal company operations
1 Personal but in professional context
1 confidentiality
1 anxiety
2 Logistic Arrangements
2 Forwarded email(s) including replies
3 Logistic Arrangements
3 Forwarded email(s) including replies
4 Logistic Arrangements
4 Forwarded email(s) including replies
5 Logistic Arrangements
5 Newsletters
5 Forwarded email(s) including replies
6 Employment arrangements
6 Newsletters
6 Forwarded email(s) including replies
6 gratitude
6 confidentiality
7 Company Business, Strategy
7 Newsletters
7 Forwarded email(s) including replies
7 alliances
8 Company Business, Strategy
8 Personal but in professional context
8 Newsletters
8 Forwarded email(s) including replies
8 alliances
8 admiration
8 confidentiality
9 Company Business, Strategy
9 internal projects
9 meeting minutes
9 confidentiality
10 Purely Personal
10 gratitude
11 Personal but in professional context
12 Purely Personal
13 Personal but in professional context
13 Newsletters
13 JokesBu

C:\Users\ag23489\.conda\envs\mypy36\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


 Company Business, Strategy
67 Forwarded email(s) including replies
67 california energy crisis
67 confidentiality
67 agitation
68 Company Business, Strategy
68 Newsletters
68 Forwarded email(s) including replies
68 california energy crisis
68 internal company policy
69 Company Business, Strategy
69 internal projects
69 company image
69 california energy crisis
69 internal company operations
69 confidentiality
69 concern
70 Personal but in professional context
70 Forwarded email(s) including replies
70 agitation
71 Company Business, Strategy
71 Newsletters
71 Forwarded email(s) including replies
71 talking points
71 confidentiality
72 Company Business, Strategy
72 Attachment(s)
72 california energy crisis
73 Logistic Arrangements
74 Company Business, Strategy
74 News article(s)
74 company image
75 Document editing/checking 
75 Attachment(s)
76 Company Business, Strategy
76 News article(s)
76 company image
76 california energy crisis
76 anxiety
77 Company Business, Strategy
77 Newslette

197 company image
197 company image
197 california energy crisis
198 Company Business, Strategy
198 Empty message
198 Forwarded email(s) including replies
198 california energy crisis
199 Company Business, Strategy
199 News article(s)
199 legal advice
199 political influence
199 california energy crisis
200 Company Business, Strategy
200 News article(s)
200 legal advice
200 company image
200 company image
200 california energy crisis
201 Company Business, Strategy
201 Newsletters
201 News article(s)
201 california energy crisis
202 Company Business, Strategy
202 News article(s)
202 company image
203 Empty message
203 News article(s)
204 Company Business, Strategy
204 Newsletters
204 News article(s)
204 california energy crisis
205 Company Business, Strategy
205 Personal but in professional context
205 Newsletters
205 Forwarded email(s) including replies
205 confidentiality
206 Company Business, Strategy
206 Document editing/checking 
206 Business letter/Document
206 internal company po

332 Attachment(s)
332 company image
333 Purely Personal
333 Logistic Arrangements
333 Forwarded email(s) including replies
334 Personal but in professional context
334 Logistic Arrangements
334 Newsletters
334 Forwarded email(s) including replies
335 Logistic Arrangements
335 Document editing/checking 
335 Newsletters
335 Forwarded email(s) including replies
336 Personal but in professional context
336 Logistic Arrangements
336 Newsletters
336 Forwarded email(s) including replies
336 confidentiality
337 Logistic Arrangements
338 Purely Personal
338 Logistic Arrangements
338 Newsletters
338 Forwarded email(s) including replies
339 Purely Personal
339 Logistic Arrangements
339 Newsletters
339 Forwarded email(s) including replies
340 Personal but in professional context
340 Logistic Arrangements
340 Newsletters
340 Forwarded email(s) including replies
341 Personal but in professional context
341 Employment arrangements
342 Logistic Arrangements
342 Newsletters
342 Forwarded email(s) inclu

...

952 Forwarded email(s) including replies
952 Attachment(s)
952 internal company operations
953 Company Business, Strategy
953 Newsletters
953 Forwarded email(s) including replies
953 internal company policy
953 internal company operations
954 Company Business, Strategy
954 Attachment(s)
954 california energy crisis
955 Employment arrangements
955 Newsletters
955 Forwarded email(s) including replies
956 Logistic Arrangements
957 Logistic Arrangements
957 Forwarded email(s) including replies
957 Attachment(s)
958 Personal but in professional context
958 Logistic Arrangements
958 Forwarded email(s) including replies
959 Company Business, Strategy
959 Forwarded email(s) including replies
959 internal projects
960 Logistic Arrangements
960 Newsletters
960 Forwarded email(s) including replies
961 Company Business, Strategy
961 Forwarded email(s) including replies
961 political influence
962 Purely Personal
962 Forwarded email(s) including replies
962 Attachment(s)
962 humor
963 Purely Person

1098 legal advice
1098 california energy crisis
1099 Company Business, Strategy
1099 Newsletters
1099 Forwarded email(s) including replies
1099 internal company operations
1100 Company Business, Strategy
1100 Newsletters
1100 News article(s)
1100 Pointers to url(s)
1100 california energy crisis
1101 Company Business, Strategy
1101 Forwarded email(s) including replies
1101 alliances
1102 Logistic Arrangements
1103 Company Business, Strategy
1103 Newsletters
1103 Forwarded email(s) including replies
1103 legal advice
1103 california energy crisis
1103 confidentiality
1104 Company Business, Strategy
1104 political influence
1104 confidentiality
1105 Logistic Arrangements
1106 Company Business, Strategy
1106 Newsletters
1106 Forwarded email(s) including replies
1106 internal company policy
1107 Logistic Arrangements
1108 Logistic Arrangements
1109 Document editing/checking 
1109 Attachment(s)
1109 confidentiality
1110 Company Business, Strategy
1110 california energy crisis
1110 legal advi

1267 Company Business, Strategy
1267 political influence
1267 california energy crisis
1267 camaraderie
1268 Logistic Arrangements
1269 Company Business, Strategy
1269 Newsletters
1269 Attachment(s)
1269 company image
1270 Empty message
1271 Company Business, Strategy
1271 Business letter/Document
1271 internal company operations
1272 Company Business, Strategy
1272 Newsletters
1272 Forwarded email(s) including replies
1272 Business letter/Document
1272 Attachment(s)
1272 internal company operations
1273 Company Business, Strategy
1273 Attachment(s)
1273 company image
1273 company image
1273 california energy crisis
1274 Company Business, Strategy
1274 Newsletters
1274 Forwarded email(s) including replies
1274 Business letter/Document
1274 Attachment(s)
1274 legal advice
1275 Logistic Arrangements
1275 Newsletters
1275 Forwarded email(s) including replies
1276 Company Business, Strategy
1276 political influence
1277 Company Business, Strategy
1277 Newsletters
1277 Forwarded email(s) in

1427 Attachment(s)
1427 internal projects
1428 Logistic Arrangements
1428 Forwarded email(s) including replies
1429 Logistic Arrangements
1429 Newsletters
1429 Forwarded email(s) including replies
1430 Company Business, Strategy
1430 Newsletters
1430 Forwarded email(s) including replies
1430 legal advice
1431 Company Business, Strategy
1431 Forwarded email(s) including replies
1431 legal advice
1431 company image
1431 internal company operations
1432 Company Business, Strategy
1432 company image
1433 Company Business, Strategy
1433 Document editing/checking 
1433 Forwarded email(s) including replies
1433 Attachment(s)
1433 company image
1433 california energy crisis
1434 Document editing/checking 
1434 Newsletters
1434 Forwarded email(s) including replies
1435 Logistic Arrangements
1435 Newsletters
1436 Personal but in professional context
1436 Forwarded email(s) including replies
1437 Company Business, Strategy
1437 Newsletters
1437 Forwarded email(s) including replies
1437 california

1569 Government / academic report(s)
1569 legal advice
1570 Company Business, Strategy
1570 Attachment(s)
1570 california energy crisis
1570 confidentiality
1571 Company Business, Strategy
1571 california energy crisis
1572 Company Business, Strategy
1572 legal advice
1572 political influence
1573 Company Business, Strategy
1573 News article(s)
1573 Government action(s)
1573 legal advice
1574 Company Business, Strategy
1574 Forwarded email(s) including replies
1574 company image
1575 Company Business, Strategy
1575 Newsletters
1575 Forwarded email(s) including replies
1575 political influence
1576 Company Business, Strategy
1576 Government action(s)
1576 legal advice
1577 Company Business, Strategy
1577 legal advice
1578 Company Business, Strategy
1578 Government action(s)
1578 legal advice
1578 legal advice
1579 Document editing/checking 
1580 Company Business, Strategy
1580 Newsletters
1580 Pointers to url(s)
1580 political influence
1581 Company Business, Strategy
1581 Forwarded ema

In [35]:
labelsWithNames

[['Company Business, Strategy', 'internal company operations'],
 ['Personal but in professional context', 'confidentiality', 'anxiety'],
 ['Logistic Arrangements', 'Forwarded email(s) including replies'],
 ['Logistic Arrangements', 'Forwarded email(s) including replies'],
 ['Logistic Arrangements', 'Forwarded email(s) including replies'],
 ['Logistic Arrangements',
  'Newsletters',
  'Forwarded email(s) including replies'],
 ['Employment arrangements',
  'Newsletters',
  'Forwarded email(s) including replies',
  'gratitude',
  'confidentiality'],
 ['Company Business, Strategy',
  'Newsletters',
  'Forwarded email(s) including replies',
  'alliances'],
 ['Company Business, Strategy',
  'Personal but in professional context',
  'Newsletters',
  'Forwarded email(s) including replies',
  'alliances',
  'admiration',
  'confidentiality'],
 ['Company Business, Strategy',
  'internal projects',
  'meeting minutes',
  'confidentiality'],
 ['Purely Personal', 'gratitude'],
 ['Personal but in pr

In [36]:
email_df_labelled[email_df_labelled['Company Business, Strategy']==1]

,Message-ID,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,...,confidentiality,anxiety,concern,competitiveness,gloating,pride,agitation,despair,shame,dislike
0,<9831685.1075855725804.JavaMail.evans@thyme>,2001-03-15 06:45:00-08:00,(phillip.allen@enron.com),(todd.burke@enron.com),Re: Confidential Employee Information/Lenhart,Phillip K Allen,Todd Burke,,,\Phillip_Allen_June2001\Notes Folders\'sent mail,...,0,0,0,0,0,0,0,0,0,0
7,<860767.1075849626951.JavaMail.evans@thyme>,2000-12-14 02:51:00-08:00,(matt.harris@enron.com),(sarah-joy.hunter@enron.com),Re: HP -- confidential internal document,Matt Harris,Sarah-Joy Hunter,"Dale Clark, Jennifer Medcalf, Patrick Tucker, ...",,\John_Arnold_Nov2001\Notes Folders\All documents,...,0,0,0,0,0,0,0,0,0,0
8,<17578964.1075849627055.JavaMail.evans@thyme>,2000-12-14 09:48:00-08:00,(matt.harris@enron.com),(patrick.tucker@enron.com),Re: HP -- confidential internal document,Matt Harris,Patrick Tucker,"Dale Clark, Jennifer Medcalf",,\John_Arnold_Nov2001\Notes Folders\All documents,...,1,0,0,0,0,0,0,0,0,0
9,<24049587.1075849626031.JavaMail.evans@thyme>,2000-12-12 08:42:00-08:00,(sarah-joy.hunter@enron.com),(matt.harris@enron.com),HP -- confidential internal document,Sarah-Joy Hunter,Matt Harris,"Patrick Tucker, Peter Goebel, Dale Clark, Jenn...",,\John_Arnold_Nov2001\Notes Folders\All documents,...,1,0,0,0,0,0,0,0,0,0
15,<12708474.1075863592189.JavaMail.evans@thyme>,2000-07-12 02:26:00-07:00,(bwoertz@caiso.com),(20participants@caiso.com),CAISO Notice - Congestion Reform Proposal - Ap...,"""Woertz, Byron"" <BWoertz@caiso.com>",ISO Market Participants <IMCEAEX-_O=CAISO_OU=C...,"""Schmid, Elena"" <ESchmid@caiso.com>, ""Robinson...",,\Robert_Badeer_Aug2000\Notes Folders\All docum...,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,<17191225.1075852348672.JavaMail.evans@thyme>,2001-09-27 14:54:24-07:00,(offley@hoover.stanford.edu),(paffairs2@hoover.stanford.edu),HOOVER INSTITUTION NEWS ADVISORY: Conference o...,Caleb Offley <Offley@hoover.stanford.edu>,Paffairs2@hoover.stanford.edu,,,\GWHALLE (Non-Privileged)\Inbox,...,0,0,0,0,0,0,0,0,0,0
1696,<9573297.1075852349319.JavaMail.evans@thyme>,2001-09-17 08:39:10-07:00,(j.kaminski@enron.com),(greg.whalley@enron.com),FW:,"Kaminski, Vince J </O=ENRON/OU=NA/CN=RECIPIENT...","Whalley, Greg </O=ENRON/OU=NA/CN=RECIPIENTS/CN...","Kaminski, Vince J </O=ENRON/OU=NA/CN=RECIPIENT...",,\GWHALLE (Non-Privileged)\Inbox,...,0,0,0,0,0,0,0,0,0,0
1697,<18361957.1075861368310.JavaMail.evans@thyme>,2001-11-05 05:42:45-08:00,(louise.kitchen@enron.com),"(john.lavorato@enron.com, w..delainey@enron.co...",Confidential Due Diligence Info - GS,"Kitchen, Louise </O=ENRON/OU=NA/CN=RECIPIENTS/...","Whalley, Greg </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",,,"\GWHALLE (Non-Privileged)\Whalley, Greg\Inbox",...,0,0,0,0,0,0,0,0,0,0
1698,<29049822.1075842045435.JavaMail.evans@thyme>,2001-06-13 07:00:00-07:00,(steven.kean@enron.com),"(jeff.skilling@enron.com, james.derrick@enron....",California's dim bulbs,Steven J Kean,"Kenneth Lay, Jeff Skilling, Andrew S Fastow, G...",,,\Lawrence_Whalley_Nov2001\Notes Folders\All do...,...,0,0,0,0,0,0,0,0,0,0


In [37]:
email_df_labelled.iloc[3][53:]

Purely Personal                                        0
Personal but in professional context                   0
Logistic Arrangements                                  1
Employment arrangements                                0
Document editing/checking                              0
Empty message Attachment                               0
Empty message                                          0
Includes new text in addition to forwarded material    0
Forwarded email(s) including replies                   1
Business letter/Document                               0
News article(s)                                        0
Government / academic report(s)                        0
Government action(s)                                   0
Press release(s)                                       0
Legal documents                                        0
Pointers to url(s)                                     0
Newsletters                                            0
JokesBusiness                  

In [51]:
listForLabelColumn = []
email_df_labelled[email_df_labelled['meeting minutes']==1]['content']

labels = []
for index,row in email_df_labelled.iterrows():
    labelsList = []
    for i in range(52,103):
        if row[i] == 1:
            labelsList.append(row.index[i])
    labels.append(labelsList)
    #email_df_labelled.loc[index,'labels'] = labels
    
    #print(row.index[104],row[104])
    

C:\Users\ag23489\.conda\envs\mypy36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [53]:
email_df_labelled['labels'] = labels

C:\Users\ag23489\.conda\envs\mypy36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [56]:
email_df_labelled['labels']

0       [Company Business, Strategy, internal company ...
1       [Personal but in professional context, confide...
2       [Logistic Arrangements, Forwarded email(s) inc...
3       [Logistic Arrangements, Forwarded email(s) inc...
4       [Logistic Arrangements, Forwarded email(s) inc...
                              ...                        
1697    [Company Business, Strategy, Attachment(s), in...
1698    [Company Business, Strategy, News article(s), ...
1699    [Employment arrangements, Forwarded email(s) i...
1700    [Company Business, Strategy, Pointers to url(s...
1701    [Purely Personal, Forwarded email(s) including...
Name: labels, Length: 1702, dtype: object

In [58]:
import pickle
output = open('enromEmailsWithLabels.pkl', 'wb')
pickle.dump(email_df_labelled,output)